In [ ]:
import csv
import pandas as pd
pd.options.plotting.backend = "plotly"
from os import listdir
from os.path import isfile, join
import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots

In [ ]:
# disable generating charts for specific benchmarks by removing them from the list
benchmarks = ['batchnorm', 'conv', 'ctfft', 'emptyloop', 'linear', 'matmul', 'primetest', 'quicksort']
speedup = False
separate_compilers = True


results_path = 'results/normal/'
CPUS = ["gold_6130", "i7_5960X", "i7_7700", "i9_7960X", "i9_9940X", "platinum_8280"]

In [ ]:
def generate_data(foldername):
    allfiles = [f"{foldername}/{f}" for f in listdir(foldername) if isfile(join(foldername, f))]
    num_cores = len(allfiles)
    mean_data = {}
    stddev_data = {}

    for i in range(1, num_cores+1):
        filename = f"{foldername}/out_{i}.csv"
        assert filename in allfiles
        results = pd.read_csv(filename, sep=';')
        for i, row in results.iterrows():
            desc = row['Descriptor']
            mode = row['Mode']
            if desc in mean_data:
                if mode in mean_data[desc]:
                    mean_data[desc][mode] += [row['Mean']]
                    stddev_data[desc][mode] += [row['StdDev']]
                else:
                    mean_data[desc].update({mode: [row['Mean']]})
                    stddev_data[desc].update({mode: [row['StdDev']]})
            else:
                mean_data[desc] = {mode: [row['Mean']]}
                stddev_data[desc] = {mode: [row['StdDev']]}

    return mean_data, stddev_data

In [ ]:
gcc_data = dict()
clang_data = dict()
icc_data = dict()
icx_data = dict()
for bench in benchmarks:
    for cpu in CPUS:
        #tuple: mean, stddev
        gcc_data[cpu] = generate_data(f"{results_path}/{cpu}/gcc_results/{bench}")
        clang_data[cpu] = generate_data(f"{results_path}/{cpu}/clang_results/{bench}")
        icc_data[cpu] = generate_data(f"{results_path}/{cpu}/icc_results/{bench}")
        icx_data[cpu] = generate_data(f"{results_path}/{cpu}/icx_results/{bench}")

for bench in benchmarks:

    descriptors = list(gcc_data['gold_6130'][0].keys()) 

    for desc in descriptors:
        for cpu in CPUS:
            assert desc in clang_data[cpu][0]
            serial_gcc = gcc_data[cpu][0][desc].pop('SERIAL', None)
            serial_clang = clang_data[cpu][0][desc].pop('SERIAL', None)
            serial_icc = icc_data[cpu][0][desc].pop('SERIAL', None)
            serial_icx = icx_data[cpu][0][desc].pop('SERIAL', None)
            serial_dev_gcc = gcc_data[cpu][1][desc].pop('SERIAL', None)
            serial_dev_clang = clang_data[cpu][1][desc].pop('SERIAL', None)
            serial_dev_icc = icc_data[cpu][1][desc].pop('SERIAL', None)
            serial_dev_icx = icx_data[cpu][1][desc].pop('SERIAL', None)
            
            gcc_results = pd.DataFrame(data=gcc_data[cpu][0][desc])
            gcc_results = gcc_results.rdiv(serial_gcc[0]) if speedup else gcc_results
            gcc_results = gcc_results.add_prefix('GCC_')
            gcc_error = pd.DataFrame(data=gcc_data[cpu][1][desc])

            clang_results = pd.DataFrame(data=clang_data[cpu][0][desc])
            clang_results = clang_results.rdiv(serial_clang[0]) if speedup else clang_results
            clang_results = clang_results.add_prefix('CLANG_')
            clang_error = pd.DataFrame(data=clang_data[cpu][1][desc])

            icc_results = pd.DataFrame(data=icc_data[cpu][0][desc])
            icc_results = icc_results.rdiv(serial_icc[0]) if speedup else icc_results
            icc_results = icc_results.add_prefix('ICC_')
            icc_error = pd.DataFrame(data=icc_data[cpu][1][desc])

            icx_results = pd.DataFrame(data=icx_data[cpu][0][desc])
            icx_results = icx_results.rdiv(serial_icx[0]) if speedup else icx_results
            icx_results = icx_results.add_prefix('ICX_')
            icx_error = pd.DataFrame(data=icx_data[cpu][1][desc])

            all_ = [gcc_results, clang_results, icc_results, icx_results]
            errors_ = [gcc_error, clang_error, icc_error, icx_error]

            styles =  ['solid', 'dash', 'dashdot', 'dot']
            markers = ['triangle-up', 'square', 'star', 'circle']
            if separate_compilers:
                fig = make_subplots(rows=2, cols=2, start_cell="top-left", subplot_titles=("GCC Results", "CLANG Results", "ICC Results", "ICX Results"))
                for i in range(len(all_)):
                    res = all_[i]
                    err = errors_[i]
                    for j in range(len(res.columns)):
                        fig.add_trace(go.Scatter(x=np.arange(1,len(res)+1), y=res.iloc[:,j], name=res.columns[j], line=dict(dash=styles[i]),
                                      error_y=None if speedup else dict(type='data', array=list(err.iloc[:,j]), visible=True),
                                      mode = 'lines+markers', marker = dict(symbol = markers[i], size=4)), row=int(i/2)+1, col=int(i%2)+1)

                fig.update_layout(title=f'{bench} - {desc} - {cpu}',
                            height=500,
                            width=800,
                            xaxis_title='Threads',
                            yaxis_title='Speedup',
                            )
                fig.show()
            else:
                fig = go.Figure()
                for i in range(len(all_)):
                    res = all_[i]
                    err = errors_[i]
                    for j in range(len(res.columns)):
                        fig.add_bar(x=np.arange(1,len(res)+1), y=res.iloc[:,j], name=res.columns[j],line=dict(dash=styles[i]),
                                        error_y=None if speedup else dict(type='data', array=list(err.iloc[:,j]), visible=True),
                                        mode = 'lines+markers', marker = dict(symbol = markers[i], size=8))
                fig.update_layout(title=f'<b>{bench} - {desc} - {cpu}</b>',
                            xaxis_title='Threads',
                            yaxis_title='Speedup',
                            height=400,
                            width=600,
                            )
                fig.show()